In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.cov_struct import (Exchangeable,
    Independence,Autoregressive)
from statsmodels.genmod.families import Poisson

#### Demographic Data Processing


In [2]:
def remove_minus(x):
    if (x == "-"):
        return "NaN"
    else:
        return x.replace("-", "").replace("+", "").replace(",", "")

In [16]:
#this data is for 11 variables, all census tracts in California, using ACS 2009 5YR estimates 

demog_data_path = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/demographic_data/'
#get percentages of each race

race = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B03002_race_clean.csv')
race["Percent NH White alone"] = race["Estimate; Not Hispanic or Latino: - White alone"]/race["Estimate; Total:"]
race["Percent NH Black or African African alone"] = race["Estimate; Not Hispanic or Latino: - Black or African American alone"]/race["Estimate; Total:"]
race["Percent NH Asian alone"] = race["Estimate; Not Hispanic or Latino: - Asian alone"]/race["Estimate; Total:"]
race["Percent Hispanic"] = race["Estimate; Hispanic or Latino:"]/race["Estimate; Total:"]


edu = pd.read_csv(demog_data_path+ 'ACS_09_5YR_S1501_bach_clean.csv')
edu = edu.rename({"Percent of population 25 years and over  with Bachelor\'s degree": "Percent of population 25 years and over with Bachelor\'s degree"}, axis=1)
edu["Percent of population 25 years and over with Bachelor\'s degree"] = edu["Percent of population 25 years and over with Bachelor\'s degree"].apply(lambda x: x.replace("+", "").replace("-", "NaN"))
edu["Percent of population 25 years and over with Bachelor\'s degree"] = edu["Percent of population 25 years and over with Bachelor\'s degree"].astype(float).dropna()


#get rid of + and ***
hvalue = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25077_hvalue_clean.csv')
hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].apply(lambda x: x.replace("+", "").replace("-", "NaN").replace(",", ""))
hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].astype(float)

#hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].astype(float).dropna()

#add attached and detached and find the %
single = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25024_singlefam_clean.csv')
single["Total Single Family"] = single["Estimate; 1, attached"]+ single["Estimate; 1, detached"]
single["% Single Family"] = single["Total Single Family"] / single["Estimate; Total:"]
#single["Tract"] = (single["Id2"].astype(str)).apply(lambda x: int(x[4:8]))

poverty = pd.read_csv(demog_data_path+"ACS_09_5YR_S1702_povertyfam_clean.csv")

# poverty = pd.read_csv(demog_data_path+ "ACS_17_5YR_S1701_poverty_clean.csv")
poverty['All families - Percent  below poverty level; Estimate; Families'] = poverty['All families - Percent  below poverty level; Estimate; Families'].apply(lambda x: x.replace("+", "").replace("-", "NaN"))
poverty['All families - Percent  below poverty level; Estimate; Families'] = poverty['All families - Percent  below poverty level; Estimate; Families'].astype(float).dropna()



units = pd.read_csv(demog_data_path+'ACS_09_5YR_B25001_units_clean.csv')
#units["Tract"] = (units["Id2"].astype(str)).apply(lambda x: int(x[4:8]))


#find % renter occupied
tenure = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25003_tenure_clean.csv')
tenure["% Renter Occupied"] = tenure["Estimate; Renter occupied"]/tenure["Estimate; Total:"]
tenure["% Owner Occupied"] = tenure["Estimate; Owner occupied"]/tenure["Estimate; Total:"]


income = pd.read_csv(demog_data_path + 'ACS_09_5YR_S1903_income.csv')
income["Median income (dollars); All households"] = income["Median income (dollars); All households"].apply(remove_minus)

income["Median income (dollars); All households"] = income["Median income (dollars); All households"].astype(float)
len(poverty)



7049

#### Create Controlled features dataframe

In [49]:
race_units_merged = race.merge(units, how='left', left_on="Id2", right_on="Id2")
print("race and units merged", "length", len(race_units_merged), "unique tracts ", len(race_units_merged['Id2'].unique()))
#print("race and units merged", "dtypes", race_units_merged.dtypes)

edu_merged = race_units_merged.merge(edu, how='left', left_on="Id2", right_on="Id2")
print("edu merged", "length", len(edu_merged), "unique tracts ", len(edu_merged['Id2'].unique()))
#print("edu merged", "dtypes", edu_merged.dtypes)

income_merged = edu_merged.merge(income, how='left', left_on="Id2", right_on="Id2")
print("income merged", "length", len(income_merged), "unique tracts ", len(income_merged['Id2'].unique()))
#print("income", "dtypes", income_merged.dtypes)

poverty_merged = income_merged.merge(poverty, how='left', left_on="Id2", right_on="Id2")
print("poverty merged", "length", len(poverty_merged), "unique tracts ", len(poverty_merged['Id2'].unique()))
#print("poverty", "dtypes", poverty_merged.dtypes)

hvalue_merged = poverty_merged.merge(hvalue, how='left', left_on="Id2", right_on="Id2")
print("hvalue merged", "length", len(hvalue_merged), "unique tracts ", len(hvalue_merged['Id2'].unique()))
#print("hvalue", "dtypes", hvalue_merged.dtypes)

single_merged = hvalue_merged.merge(single, how='left', left_on="Id2", right_on="Id2")
print("single merged", "length", len(single_merged), "unique tracts ", len(single_merged['Id2'].unique()))
#print("singe merged", "dtypes", single_merged.dtypes)


merged = single_merged.merge(tenure, how='left', left_on="Id2", right_on="Id2")
print("tenure merged", "length", len(merged), "unique tracts ", len(merged['Id2'].unique()))
merged.head()

race and units merged length 7049 unique tracts  7049
edu merged length 7049 unique tracts  7049
income merged length 7049 unique tracts  7049
poverty merged length 7049 unique tracts  7049
hvalue merged length 7049 unique tracts  7049
single merged length 7049 unique tracts  7049
tenure merged length 7049 unique tracts  7049


,Id2,Geography_x,Estimate; Total:_x,Margin of Error; Total:_x,Estimate; Not Hispanic or Latino:,Estimate; Not Hispanic or Latino: - White alone,Estimate; Not Hispanic or Latino: - Black or African American alone,Estimate; Not Hispanic or Latino: - Asian alone,Estimate; Hispanic or Latino:,Estimate; Hispanic or Latino: - White alone,...,% Single Family,Geography_y,Estimate; Total:,Margin of Error; Total:,Estimate; Owner occupied,Margin of Error; Owner occupied,Estimate; Renter occupied,Margin of Error; Renter occupied,% Renter Occupied,% Owner Occupied
0,6001400100,"Census Tract 4001, Alameda County, California",2872,233,2851,2209,119,318,21,15,...,0.912439,"Census Tract 4001, Alameda County, California",1308,108,1171,107,137,74,0.104740,0.895260
1,6001400200,"Census Tract 4002, Alameda County, California",2076,165,1854,1594,47,144,222,56,...,0.665236,"Census Tract 4002, Alameda County, California",902,42,593,71,309,73,0.342572,0.657428
2,6001400300,"Census Tract 4003, Alameda County, California",4964,369,4541,3545,492,375,423,247,...,0.478758,"Census Tract 4003, Alameda County, California",2626,132,1064,150,1562,182,0.594821,0.405179
3,6001400400,"Census Tract 4004, Alameda County, California",4223,553,3957,2948,391,330,266,149,...,0.555446,"Census Tract 4004, Alameda County, California",1964,87,852,133,1112,152,0.566191,0.433809
4,6001400500,"Census Tract 4005, Alameda County, California",3759,308,3323,1577,1303,249,436,179,...,0.444380,"Census Tract 4005, Alameda County, California",1686,82,738,102,948,119,0.562278,0.437722


In [53]:
merged.to_csv('census09.csv')

In [52]:
merged.columns
merged = merged[['Id2','Percent of population 25 years and over with Bachelor\'s degree', 
      'All families - Percent  below poverty level; Estimate; Families', 
      '% Single Family', '% Owner Occupied', 'Percent NH White alone',
       'Percent NH Black or African African alone', 'Percent NH Asian alone',
       'Percent Hispanic', 'Estimate; Total Number of Housing Units', 'Estimate; Median value (dollars)', 
        'Median income (dollars); All households']]
merged_rename = merged.rename({'All families - Percent  below poverty level; Estimate; Families': "% below poverty level",
              'Estimate; Total Number of Housing Units': 'Total number of housing units',
              'Estimate; Median value (dollars)': "Median home value",
              'Median income (dollars); All households': "Median income", 'Id2': "Geoid"}, axis=1)
#print("num tracts", len(merged))
      
merged_rename




,Geoid,Percent of population 25 years and over with Bachelor's degree,% below poverty level,% Single Family,% Owner Occupied,Percent NH White alone,Percent NH Black or African African alone,Percent NH Asian alone,Percent Hispanic,Total number of housing units,Median home value,Median income
0,6001400100,34.5,3.1,0.912439,0.895260,0.769150,0.041435,0.110724,0.007312,1439,1000000.0,186439.0
1,6001400200,37.6,0.0,0.665236,0.657428,0.767823,0.022640,0.069364,0.106936,932,909500.0,122647.0
2,6001400300,32.1,6.9,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,718100.0,66638.0
3,6001400400,44.0,4.0,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,790500.0,80391.0
4,6001400500,28.1,6.0,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,572000.0,50658.0
5,6001400600,51.9,26.4,0.632653,0.470756,0.338305,0.473150,0.032220,0.059666,784,586700.0,39802.0
6,6001400700,19.4,8.1,0.572421,0.395335,0.356832,0.453986,0.095104,0.049218,2016,511600.0,32471.0
7,6001400800,33.5,21.7,0.423123,0.297976,0.326789,0.378702,0.174709,0.086190,1678,499100.0,37750.0
8,6001400900,28.2,4.0,0.492487,0.438356,0.376453,0.545182,0.007499,0.041245,1198,524600.0,51767.0
9,6001401000,19.0,31.7,0.466443,0.423432,0.166775,0.566320,0.143205,0.071196,2682,415600.0,36875.0


#### Get only Bay Area counties

In [30]:
county_codes = ['6001', '6013', '6041', '6055', '6075', '6081', '6085', '6097', '6095']


In [31]:
def foo(x):
    s = str(x)[:4]
    #print(s)
    return s

merged_bay = merged[merged["Id2"].apply(foo).isin(county_codes)]
print("num tracts", len(merged_bay))


num tracts 1405


#### Merge Demographic features with neighbors dataframe to contribute CRA eligibility information by tract and exclude CTs without oppositely coded neighbors

Universe: all of the CTs in the 9 county bay area that had loan activity. The tracts_type_master.csv file includes all the tracts for every year and their CRA eligibility, here the 2009 entries from that file are merged with the bay tracts to include tracts and their CRA eligibilities.



In [73]:
master = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/reference/tracts_type_master_1.csv')
master_09 = master[master["Year"] == 2009]
master_09.head() #1395 tracts in the master reference
#master_09[master_09['County']== 'San Francisco County']
# len(master_09['Geoid'].unique())
master_09[master_09["County"] == 'San Francisco County']
master_09[master_09["County"] == 'Sonoma County']



,Geoid,Tract,Year,type,County,CRA Eligible
3036,6097150100,1501.00,2009,mid,Sonoma County,not eligible
3037,6097150201,1502.01,2009,mid,Sonoma County,not eligible
3038,6097150202,1502.02,2009,mid,Sonoma County,not eligible
3039,6097150302,1503.02,2009,mod,Sonoma County,eligible
3040,6097150303,1503.03,2009,mid,Sonoma County,not eligible
3041,6097150304,1503.04,2009,mid,Sonoma County,not eligible
3042,6097150500,1505.00,2009,mid,Sonoma County,not eligible
3043,6097150601,1506.01,2009,mid,Sonoma County,not eligible
3044,6097150602,1506.02,2009,mid,Sonoma County,not eligible
3045,6097150603,1506.03,2009,mid,Sonoma County,not eligible


In [67]:
merged_bay[~merged_bay['Id2'].isin(master_09['Geoid'])] #203 in the all bay tracts that are not in the HMDA tracts


203

In [70]:
foo = master_09[~(master_09['Geoid'].isin(merged_bay['Id2']))] #179 in the HMDA 2009 tracts that are not in the census tracts
#foo.to_csv('inhmda09_notinbaycensus.csv')
foo["County"].unique()


array(['San Francisco County', 'Santa Clara County'], dtype=object)

In [27]:
demog_with_cra = pd.merge(merged_bay, master_09, how='right', left_on='Id2', right_on='Geoid').drop("Geoid", axis=1) 
#i think that this is supposed to be a right merge because the demographics dataframe has all the census tracts, while the boolean neighbors one only
#has the census tracts in the bay area that have neighbors
demog_with_cra = demog_with_cra.drop(["Tract", "Year"], axis=1)
len(demog_with_cra) # these are the tracts in 
#demog_with_cra


1395

#### Get total number and percentages of loans data from HMDA files
This is currently for 2009, but next step is to do it for the average of the recovery period 2013-2017


Universe: all of the CTs in the 9 county bay area that had loan activity during 2009

In [21]:
filepath = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/parsed_data/'
Alameda = pd.read_csv(filepath+'Alameda_2009_parsed.csv')
Alameda["Geoid"] = Alameda["Tract"].apply(lambda x: int(x*100 + 6001000000))

ContraCosta = pd.read_csv(filepath+'ContraCosta_2009_parsed.csv')
ContraCosta["Geoid"] = ContraCosta["Tract"].apply(lambda x: int(x*100 + 6013000000))

Napa = pd.read_csv(filepath+'Napa_2009_parsed.csv')
Napa["Geoid"] = Napa["Tract"].apply(lambda x: int(x*100 + 6055000000))

Marin = pd.read_csv(filepath+'Marin_2009_parsed.csv')
Marin["Geoid"] = Marin["Tract"].apply(lambda x: int(x*100 + 6041000000))

SanMateo = pd.read_csv(filepath+'SanMateo_2009_parsed.csv')
SanMateo["Geoid"] = SanMateo["Tract"].apply(lambda x: int(x*100 + 6081000000))

SanFrancisco = pd.read_csv(filepath+'SanFrancisco_2009_parsed.csv')
SanFrancisco["Geoid"] = SanFrancisco["Tract"].apply(lambda x: int(x*100 + 6075000000))

Solano = pd.read_csv(filepath+'Solano_2009_parsed.csv')
Solano["Geoid"] = Solano["Tract"].apply(lambda x: int(x*100 + 6095000000))

SantaClara = pd.read_csv(filepath+'SantaClara_2009_parsed.csv')
SantaClara["Geoid"] = SantaClara["Tract"].apply(lambda x: int(x*100 + 6085000000))

Sonoma = pd.read_csv(filepath+'Sonoma_2009_parsed.csv')
Sonoma["Geoid"] = Sonoma["Tract"].apply(lambda x: int(x*100 + 6097000000))

counties = [Alameda, ContraCosta, Napa, Marin, SanMateo, SanFrancisco,Solano,SantaClara, Sonoma]

bay_counties_loans = pd.concat(counties).reset_index(drop=True)
bay_counties_loans["Total Loans"] = bay_counties_loans['# HI borrower, LI tract'] + bay_counties_loans['# HI borrower, HI tract']+ bay_counties_loans['# LI borrower, LI tract']
+ bay_counties_loans['# LI borrower, HI tract']
bay_counties_loans = bay_counties_loans.drop(["Tract", "Year"], axis=1)


#### Merge Loans Data with cra_demog features

In [23]:
allvars_2009 = pd.merge(demog_with_cra, bay_counties_loans, how="left", left_on='Id2', right_on='Geoid').drop(['CRA Eligible_y', "Geoid"], axis=1)
allvars_2009['CRA'] = allvars_2009['CRA Eligible_x'].apply(lambda x: 1 if x == 'eligible' else 0)
allvars_2009.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/all_vars2009.csv")

len(allvars_2009)



1406

#### Regression Data Prep

In [26]:
allvars_2009.columns
noinc = ['% HI borrower, LI tract',
       '# HI borrower, LI tract', '% HI borrower, HI tract',
       '# HI borrower, HI tract', '% LI borrower, LI tract',
       '# LI borrower, LI tract', '%LI borrower, HI tract',
       '# LI borrower, HI tract', 'Geoid', 'Total Loans', "CRA Eligible_x", "type_x", "County_x", "County_y", 
         "type_y", "Id2"]

X_vars = allvars_2009.loc[:, ~allvars_2009.columns.isin(noinc)]
y = allvars_2009["Total Loans"]
print(len(y))
print(len(X_vars))
X_vars.head()

1406
1406


,Percent of population 25 years and over with Bachelor's degree,All families - Percent below poverty level; Estimate; Families,% Single Family,% Owner Occupied,Percent NH White alone,Percent NH Black or African African alone,Percent NH Asian alone,Percent Hispanic,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA
0,34.5,3.1,0.912439,0.895260,0.769150,0.041435,0.110724,0.007312,1439,1000000.0,186439.0,0
1,37.6,0.0,0.665236,0.657428,0.767823,0.022640,0.069364,0.106936,932,909500.0,122647.0,0
2,32.1,6.9,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,718100.0,66638.0,0
3,44.0,4.0,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,790500.0,80391.0,0
4,28.1,6.0,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,572000.0,50658.0,1


In [27]:
#need to concatenate x with y, dropna and then unconcatenate in order to do this correctly
xydf = X_vars.copy()
xydf["y"] = y
dropped = xydf.dropna()

X_drop = dropped.loc[:, dropped.columns != 'y']
y_drop = dropped['y']

print("num tracts", len(X_drop))
print("y", len(y_drop))

print("before drop", len(xydf))
print("pct lost", (len(xydf) - len(X_drop))/len(xydf))

num tracts 1382
y 1382
before drop 1406
pct lost 0.017069701280227598


#### Data Normalization

In [31]:
# def normalize_columns(data, mean_df, std_df):
#     '''
#     Input:
#       data (data frame): contains only numeric columns
#     Output:
#       data frame, the same data, except each column is standardized 
#       to have 0-mean and unit variance
#     '''
#     normalized_data=(data-mean_df.mean())/std_df.std()

#     return normalized_data

In [32]:
# def unnormalize_columns(data, mean_df, std_df):
#     unnormalized_data=(data*std_df.std())+mean_df.mean()

#     return unnormalized_data

In [34]:
#normalize the data before perfoming the regression
# normal_x_drop = normalize_columns(X_drop, X_drop, X_drop)
# normal_y_drop = normalize_columns(y_drop, y_drop, y_drop)


#### Poisson Model(s)

In [30]:
import statsmodels.formula.api as smf

data = X_drop.copy()
data = sm.add_constant(data, prepend=False)
data.rename({'Percent NH Black or African African alone': "NHBLK", 'Percent NH White alone': 'NHWHITE', 
             'Percent NH Asian alone': 'NHASIAN', 'Percent Hispanic': 'HISPANIC', 
             'Percent of population 25 years and over with Bachelor\'s degree': 'BACHELORS', 
             'All families - Percent  below poverty level; Estimate; Families': 'POVERTYRT',
             '% Single Family': 'PCTSINGLE', '% Owner Occupied': 'PCTOWNER',
             'Estimate; Total Number of Housing Units': 'UNITS',
             'Estimate; Median value (dollars)': 'MEDVAL', 'Median income (dollars); All households': 'MEDINC',            
            }, axis=1, inplace=True)

features = list(data.columns)
features
data['y'] = y_drop

#formula = 'y ~ ' + ' + '.join(features)
formula = 'y ~ NHBLK + CRA + BACHELORS + PCTSINGLE + PCTOWNER + POVERTYRT + NHWHITE + NHBLK + NHASIAN + UNITS + NHASIAN + HISPANIC + MEDVAL + MEDINC + const' 


poisson_model_all = smf.glm(formula=formula, data=data, family=sm.families.Poisson()).fit()
poisson_model_all.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                 1382
Model:                            GLM   Df Residuals:                     1369
Model Family:                 Poisson   Df Model:                           12
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -19363.
Date:                Sun, 20 Jan 2019   Deviance:                       30408.
Time:                        23:23:47   Pearson chi2:                 3.69e+04
No. Iterations:                     6   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8271      0.057     31.952      0.000       1.715       1.939
NHBLK         -0.3870      0.127     -3.038      0.002      -0.637      -0.137
CRA            0.2169      0.012     18.204      0.000       0.194       0.240
BACHELORS      0.0067      0.001     11.269      0.000       0.006       0.008
PCTSINGLE      0.0387      0.026      1.503      0.133      -0.012       0.089
PCTOWNER       0.7075      0.034     20.584      0.000       0.640       0.775
POVERTYRT     -0.0043      0.001     -6.491      0.000      -0.006      -0.003
NHWHITE       -0.9595      0.120     -8.027      0.000      -1.194      -0.725
NHASIAN       -0.8071      0.120     -6.731      0.000      -1.042      -0.572
UNITS          0.0003   1.39e-06    221.759      0.000       0.000       0.000
HISPANIC       0.4295      0.119      3.594      0.000       0.195       0.664
MEDVAL     -7.472e-07   2.69e-08    -27.737      0.000      -8e-07   -6.94e-07
MEDINC      6.509e-06    1.8e-07     36.197      0.000    6.16e-06    6.86e-06
const          1.8271      0.057     31.952      0.000       1.715       1.939
==============================================================================
"""

In [31]:
with open('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/poisson_regression_results_all_tracts.html', 'w') as outfile:
    outfile.write(poisson_model_all.summary().as_html())
      
  

In [ ]:
# import statsmodels.formula.api as smf

# data = X_drop.copy()
# data = sm.add_constant(data, prepend=False)
# data.rename({'Percent NH Black or African African alone': "NHBLK", 'Percent NH White alone': 'NHWHITE', 
#              'Percent NH Asian alone': 'NHASIAN', 'Percent Hispanic': 'HISPANIC', 
#              'Percent of population 25 years and over with Bachelor\'s degree': 'BACHELORS', 
#              'All families - Percent  below poverty level; Estimate; Families': 'POVERTYRT',
#              '% Single Family': 'PCTSINGLE', '% Owner Occupied': 'PCTOWNER',
#              'Estimate; Total Number of Housing Units': 'UNITS',
#              'Estimate; Median value (dollars)': 'MEDVAL', 'Median income (dollars); All households': 'MEDINC',            
#             }, axis=1, inplace=True)

# features = list(data.columns)
# features
# data['y'] = y_drop

# #formula = 'y ~ ' + ' + '.join(features)
# formula = 'y ~ NHBLK + BACHELORS + PCTSINGLE + PCTOWNER + POVERTYRT + NHWHITE + NHBLK + NHASIAN + UNITS + NHASIAN + HISPANIC + MEDVAL + MEDINC + const' 


# poisson_model = smf.glm(formula=formula, data=data, family=sm.families.Poisson()).fit()
# poisson_model.summary()